# Эксперименты

In [ ]:
import numpy as np
import scipy.linalg as spl
import scipy.optimize as sopt

In [ ]:
import torch
from torch.autograd.functional import jacobian

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

font = {'family' : 'Liberation Sans',
        'weight' : 'normal',
        'size'   : 30}

matplotlib.rc('font', **font)

In [ ]:
class Immunity:
    """
    Простейшая модель иммунитета.
    """
    
    # Параметры
    a = 1.0e-7   # Скорость размножения антигена.
    b = 1.0e-5   # Скорость уничтожения антигенов.
    c = 4.0e-7   # Скорость производства специфичности.
    d = 1.0e-6   # Скорость восстановления уровня специфичности.
    
    C = 1000.0 # Предельная концентрация антигена.
    S_0 = 1.0  # Равновесная специфичность.
    
    
    def value_(self, x):
        """
        Правая часть уравнения.
        """
        
        result = torch.zeros_like(x)
        result[:,0] = self.a * x[:,0] * (self.C - x[:,0]) - self.b * x[:,0] * x[:,1]
        result[:,1] = self.c * x[:,0] - self.d * (x[:,1] - self.S_0)
        
        return result
    
    def value(self, x):
        x = torch.Tensor(x)
        return self.value_(x).numpy()
    
    def derivative(self, x):
        x = torch.Tensor(x)
        return jacobian(self.value_, x).numpy()

In [ ]:
x_0 = np.array([0.1, 2.0])
rhs = Immunity()

n_days = 10
time = n_days * 24 * 60 * 60
N = n_days * 24
dt = time / N

In [ ]:
T = np.linspace(0.0, time, N+1)
X = [x_0]

for step in range(N):
    X.append(X[-1] + dt * rhs.value(X[-1][None,:])[0])

In [ ]:
X = np.array(X)

In [ ]:
fig, ax = plt.subplots()

fig.set_figheight(12)
fig.set_figwidth(24)
ax.grid(color='#000000', alpha=0.15, linestyle='-', linewidth=1, which='major')
ax.grid(color='#000000', alpha=0.1, linestyle='-', linewidth=0.5, which='minor')

ax.set_xlabel("$t, \\; с$")
ax.set_ylabel("$x(t)$")

ax.set_yscale('log')

ax.plot(T, X[:,0], label="$ V $")
ax.plot(T, X[:,1], label="$ S $")
    
ax.legend(loc='upper left')
plt.show();